In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
test_data_url = "https://www.dropbox.com/scl/fi/qqk1277tmblj0z8r7p5xu/sample_data_dec2023.zip?rlkey=ck9zphdhyz6sco0do1gm01etj&dl=0"

In [ ]:
!if [ ! -f  {repo_root}/test_data_v1_dl.zip ] ; then wget '{test_data_url}' -O {repo_root}/test_data_v1_dl.zip ; fi 
!if [ ! -d {repo_root}/test_data/v1 ] ; then mkdir -p {repo_root}/test_data/v1; unzip {repo_root}/test_data_v1_dl.zip -d {repo_root}/test_data/v1 ; fi 

In [ ]:
import sys

sys.path.append(repo_root)  # go to parent dir
from spf.dataset.spf_dataset import SessionsDatasetRealV1

In [ ]:
data_root_dir = "%s/test_data/v1/" % repo_root
snapshots_in_file = 400000
n_thetas = 65
snapshots_in_session = 1024
width = 3000  # height is assumed to be same
step_size = 128

In [ ]:
ds = SessionsDatasetRealV1(
    root_dir=data_root_dir,
    snapshots_in_file=snapshots_in_file,
    nthetas=n_thetas,
    snapshots_in_session=snapshots_in_session,
    nsources=1,
    width=width,
    step_size=step_size,
)

In [ ]:
# pick a session from the dataset
session_idx = 1000
session = ds[session_idx]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spf.rf import get_peaks_for_2rx

idx = 490
bf_out = session["beam_former_outputs_at_t"][idx]
peaks = get_peaks_for_2rx(bf_out)
plt.plot(bf_out, color="red", label="beamformer")
for peak in peaks:
    plt.axvline(x=peak, label="Peak")
plt.legend()

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(16, 8))

axs[0, 0].invert_yaxis()  # y goes down
axs[0, 0].invert_xaxis()  # x goes left
axs[0, 0].set_xlabel("X position")
axs[0, 0].set_ylabel("Y position")
axs[0, 0].plot(
    session["source_positions_at_t"][:, 0, 0], session["source_positions_at_t"][:, 0, 1]
)
axs[0, 0].set_title("2D Position (x/y)")

axs[1, 0].plot(session["source_positions_at_t"][:, 0, 0], label="X")
axs[1, 0].plot(session["source_positions_at_t"][:, 0, 1], label="Y")
axs[1, 0].legend()
axs[1, 0].set_title("Position (x/y) vs time")
axs[1, 0].set_xlabel("Time")

axs[1, 1].plot(session["detector_orientation_at_t"] + session["source_theta_at_t"])
axs[1, 1].set_ylabel("Theta")
axs[1, 1].set_xlabel("Time")
axs[1, 1].set_title("Ground truth theta")

# plot the beamformer output
peaks_at_t = np.array(
    [get_peaks_for_2rx(bf_out) for bf_out in session["beam_former_outputs_at_t"]]
)
peaks_at_t_in_radians = (
    2 * (peaks_at_t / session["beam_former_outputs_at_t"].shape[1] - 0.5) * np.pi
)
peaks_at_t_in_radians_adjusted = (
    peaks_at_t_in_radians + 20 * session["detector_orientation_at_t"]
)  # % (np.pi*2)

axs[0, 1].plot(peaks_at_t_in_radians_adjusted[:, 0])
axs[0, 1].plot(peaks_at_t_in_radians_adjusted[:, 1])
axs[0, 1].set_xlabel("Time")
axs[0, 1].set_ylabel("Theta")

f.tight_layout()